In [1]:
import os

# Prepend the folder with espeak.exe to PATH
os.environ["PATH"] = r"C:\Program Files (x86)\eSpeak\command_line\command_line;" + os.environ["PATH"]


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate

from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=1026)
           2	LOAD_FAST(arg=0, lineno=1026)
           4	LOAD_CONST(arg=1, lineno=1026)
           6	BINARY_SUBSCR(arg=None, lineno=1026)
           8	LOAD_FAST(arg=0, lineno=1026)
          10	LOAD_CONST(arg=2, lineno=1026)
          12	BINARY_SUBSCR(arg=None, lineno=1026)
          14	COMPARE_OP(arg=4, lineno=1026)
          16	LOAD_FAST(arg=0, lineno=1026)
          18	LOAD_CONST(arg=1, lineno=1026)
          20	BINARY_SUBSCR(arg=None, lineno=1026)
          22	LOAD_FAST(arg=0, lineno=1026)
          24	LOAD_CONST(arg=3, lineno=1026)
          26	BINARY_SUBSCR(arg=None, lineno=1026)
          28	COMPARE_OP(arg=5, lineno=1026)
          30	BINARY_AND(arg=None, lineno=1026)
          32	RETURN_VALUE(arg=None, lineno=1026)
DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=102

## LJ Speech

In [3]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [4]:
import torch
print(torch.cuda.is_available())


False


In [5]:

import sys
sys.path.append(r"C:\Users\MIKKY\vits-main\vits-main")

import torch
from models import SynthesizerTrn
import utils

device = "cpu"

# Initialize model
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model
)
net_g.to(device)
net_g.eval()

# Load checkpoint (CPU-only)
checkpoint_path = r"C:\Users\MIKKY\vits-main\vits-main\pretrained_ljs.pth"
checkpoint = torch.load(checkpoint_path, map_location=device)
net_g.load_state_dict(checkpoint['model'])  # Use 'model' key

print("Model loaded successfully on CPU!")



C:\Users\MIKKY\AppData\Roaming\Python\Python38\site-packages\torch\nn\utils\weight_norm.py:134: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
C:\Users\MIKKY\AppData\Local\Temp\ipykernel_18240\3954557212.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you

Model loaded successfully on CPU!


In [6]:
import os
os.system("espeak --version")


0

In [7]:
from phonemizer import phonemize

text = "Good evening"
print(phonemize(text, language='en-us', backend='espeak'))


ɡʊd iːvnɪŋ 


In [8]:
# this is the original audio 

# 1️⃣ Convert text → sequence
stn_tst = get_text("This is awesome", hps)

# 2️⃣ Prepare input tensors
x_tst = stn_tst.unsqueeze(0).to(device)  
x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)

# ✅ Provide speaker ID if your model has multiple speakers
if net_g.n_speakers > 0:
    sid = torch.LongTensor([0]).to(device)  # use 0 or any valid speaker index
else:
    sid = None

# 3️⃣ Inference
with torch.no_grad():
    audio = net_g.infer(
        x_tst,
        x_tst_lengths,
        sid=sid,            # pass it here
        noise_scale=.667,
        noise_scale_w=0.8,
        length_scale=1
    )[0][0,0].cpu().numpy()

# 4️⃣ Play or save audio
import IPython.display as ipd
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))


logw: tensor([[[-0.5448, -0.2903,  1.8038,  0.8710,  0.2851,  1.4688,  0.3288,
           1.8502,  1.2296,  0.4546,  1.0874, -0.5412, -0.5770,  1.0194,
           1.1678,  0.4013,  1.4852,  1.4683,  1.6041,  0.5927,  1.3877,
           0.5609,  0.5055,  1.9435,  1.4757,  1.5633,  2.3689]]])
after multiplying with x_mask: tensor([[[ 0.5800,  0.7481,  6.0727,  2.3894,  1.3299,  4.3439,  1.3893,
           6.3610,  3.4198,  1.5755,  2.9665,  0.5821,  0.5616,  2.7716,
           3.2149,  1.4938,  4.4159,  4.3418,  4.9732,  1.8088,  4.0055,
           1.7522,  1.6579,  6.9829,  4.3742,  4.7747, 10.6859]]])
w tensor([[[ 0.5800,  0.7481,  6.0727,  2.3894,  1.3299,  4.3439,  1.3893,
           6.3610,  3.4198,  1.5755,  2.9665,  0.5821,  0.5616,  2.7716,
           3.2149,  2.6888,  4.4159,  7.8152,  4.9732,  3.2558,  4.0055,
           3.1539,  1.6579, 12.5692,  4.3742,  8.5945, 10.6859]]])
w_ceil: tensor([[[ 1.,  1.,  7.,  3.,  2.,  5.,  2.,  7.,  4.,  2.,  3.,  1.,  1.,  3.,
           4., 

approach A: rhythm division according to phonemes - failed 

In [21]:

def infer_with_stress(self, x, x_lengths, sid=None, noise_scale=1.0, length_scale=1.0,
                      noise_scale_w=1.0, stress_indices=None, stress_length_scale=1.0):
    """
    VITS inference with optional stress applied to selected phonemes.

    Args:
        x: Input phoneme sequence [B, T]
        x_lengths: Lengths of input sequences
        sid: Speaker ID (optional)
        noise_scale: Noise scale for latent z
        length_scale: Global length scaling
        noise_scale_w: Noise scale for duration predictor
        stress_indices: List of phoneme indices to stress
        stress_length_scale: Factor to exaggerate durations of stress_indices

    Returns:
        audio: Generated waveform [B, 1, T]
        m_p, logs_p, None, None, attn: VITS-compatible return values
    """


    
    # --- Encoder ---(mean and var)
    x, m_p, logs_p, x_mask = self.enc_p(x, x_lengths)

    # --- Duration prediction ---
    logw = self.dp(x, x_mask, reverse=True, noise_scale=noise_scale_w)  # log duration
    w = torch.exp(logw) * x_mask  # durations

    # --- Apply stress scaling only to selected indices ---
    if stress_indices is not None and stress_length_scale != 1.0:
        for idx in stress_indices:
            if idx < w.size(2):
                w[:, :, idx] *= stress_length_scale

    # --- Apply global length scale ---
    w_ceil = torch.ceil(w) * length_scale

    # --- Compute output mask ---
    y_lengths = torch.clamp_min(torch.sum(w_ceil, [1, 2]), 1).long()
    y_mask = torch.unsqueeze(commons.sequence_mask(y_lengths, None), 1).to(x_mask.dtype)

    # --- Generate attention/alignment ---
    attn_mask = torch.unsqueeze(x_mask, 2) * torch.unsqueeze(y_mask, -1)
    attn = commons.generate_path(w_ceil, attn_mask)

    # --- Project prior to latent space ---
    m_p = torch.matmul(attn.squeeze(1), m_p.transpose(1, 2)).transpose(1, 2)
    logs_p = torch.matmul(attn.squeeze(1), logs_p.transpose(1, 2)).transpose(1, 2)

    # --- Latent sampling ---
    z_p = (m_p + torch.randn_like(m_p) * torch.exp(logs_p) * noise_scale) * y_mask
    z = self.flow(z_p, y_mask, reverse=True)

    # --- Decoder ---
    audio = self.dec(z * y_mask)

    # --- Return VITS-compatible outputs ---
    return audio, m_p, logs_p, None, None, attn


In [20]:
from phonemizer import phonemize

text = "Good evening everyone! How are you? VITS is awesome!"
phonemes_text = phonemize(text, language='en-us', backend='espeak')
print(phonemes_text)
phonemes_list = phonemes_text.split()  
print(phonemes_list)


ɡʊd iːvnɪŋ ɛvɹɪwʌn haʊ ɑːɹ juː vɪts ɪz ɔːsʌm 
['ɡʊd', 'iːvnɪŋ', 'ɛvɹɪwʌn', 'haʊ', 'ɑːɹ', 'juː', 'vɪts', 'ɪz', 'ɔːsʌm']


In [22]:
word = "awesome"
word_phonemes = phonemize(word, language='en-us', backend='espeak').split()
print(word_phonemes)  # ['ɔːsʌm']


['ɔːsʌm']


In [23]:
def find_phoneme_indices(sentence_phonemes, word_phonemes):
    for i in range(len(sentence_phonemes) - len(word_phonemes) + 1):
        if sentence_phonemes[i:i+len(word_phonemes)] == word_phonemes:
            return list(range(i, i+len(word_phonemes)))
    return []

stress_indices = find_phoneme_indices(phonemes_list, word_phonemes)
print(stress_indices) 


[8]


In [24]:
# define the stress wrapper outside models.py
from types import MethodType

net_g.infer_with_stress = MethodType(infer_with_stress, net_g)

with torch.no_grad():
    audio = net_g.infer_with_stress(
        x_tst,
        x_tst_lengths,
        noise_scale = 1,
        noise_scale_w= 2,
        length_scale=1,
        stress_indices= stress_indices,   # positions for "awesome"
        stress_length_scale= 1
    )[0][0,0].cpu().numpy()

ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))


going with option B - Synthesize Stress Word Separately - failed as it is not natural.



In [66]:
# going with option B:
text = "Good evening! How are You ? VITS is awesome!"
stress_word = "awesome"

before_word = text.split(stress_word)[0].strip()  # "Good evening!  How are You ? VITS is"
after_word = text.split(stress_word)[1].strip()   # "!" (or remaining words)


In [67]:
# Whole sentence → phonemes
stn_before = get_text(before_word, hps)
stn_stress = get_text(stress_word, hps)
stn_after = get_text(after_word, hps) if after_word else None

x_lengths_before = torch.LongTensor([stn_before.size(0)])
x_lengths_stress = torch.LongTensor([stn_stress.size(0)])
x_lengths_after = torch.LongTensor([stn_after.size(0)]) if stn_after is not None else None


In [ ]:

sid = torch.LongTensor([0]).to(device)  # choose speaker 0

with torch.no_grad():
    # Before word
    audio_before = net_g.infer(
        stn_before.unsqueeze(0).to(device),
        x_lengths_before.to(device),
        sid=sid,
        noise_scale=1.0,
        noise_scale_w=0.8,
        length_scale=1.0
    )[0][0,0].cpu().numpy()

    # Stress word
    audio_stress = net_g.infer_with_stress(
        stn_stress.unsqueeze(0).to(device),
        x_lengths_stress.to(device),
        sid=sid,
        noise_scale= 1.0,
        noise_scale_w= 1.5,
        length_scale=1.0,
        stress_indices=list(range(stn_stress.size(0))),  # all phonemes
        stress_length_scale= 0.2 # exaggerate duration
    )[0][0,0].cpu().numpy()

    # After word (if any)
    audio_after = None
    if stn_after is not None:
        audio_after = net_g.infer(
            stn_after.unsqueeze(0).to(device),
            x_lengths_after.to(device),
            sid=sid,
            noise_scale=1.0,
            noise_scale_w=0.8,
            length_scale=1.0
        )[0][0,0].cpu().numpy()


In [69]:
import numpy as np

final_audio = audio_before
final_audio = np.concatenate([final_audio, audio_stress])
if audio_after is not None:
    final_audio = np.concatenate([final_audio, audio_after])
import IPython.display as ipd
ipd.display(ipd.Audio(final_audio, rate=hps.data.sampling_rate))


## VCTK

In [38]:
hps = utils.get_hparams_from_file("./configs/vctk_base.json")

In [39]:
device = torch.device("cpu")  # CPU-only

# Initialize model on CPU
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model
).to(device)
_ = net_g.eval()

# Load the pretrained VCTK checkpoint
checkpoint_path = r"C:\Users\MIKKY\vits-main\vits-main\pretrained_vctk.pth"
_ = utils.load_checkpoint(checkpoint_path, net_g, None)


c:\Users\MIKKY\vits-main\vits-main\utils.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(checkpoint_path, map_location='cpu')


INFO:root:Loaded checkpoint 'C:\Users\MIKKY\vits-main\vits-main\pretrained_vctk.pth' (iteration 0)


In [72]:
# pick device
device = torch.device("cpu")

# convert text → tensor
stn_tst = get_text("Good evening everyone. How are You ? VITS is Awesome!", hps)

with torch.no_grad():
    # send to CPU
    x_tst = stn_tst.unsqueeze(0).to(device)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).to(device)
    sid = torch.LongTensor([4]).to(device)  # target speaker ID

    # inference
    audio = net_g.infer(
        x_tst,
        x_tst_lengths,
        sid=sid,
        noise_scale=.667,
        noise_scale_w=0.8,
        length_scale=1
    )[0][0,0].cpu().float().numpy()

# play audio
import IPython.display as ipd
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=False))


### Voice Conversion

In [16]:
dataset = TextAudioSpeakerLoader(hps.data.validation_files, hps.data)
collate_fn = TextAudioSpeakerCollate()
loader = DataLoader(dataset, num_workers=8, shuffle=False,
    batch_size=1, pin_memory=True,
    drop_last=True, collate_fn=collate_fn)
data_list = list(loader)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'DUMMY2/p234/p234_071.wav'

In [ ]:
with torch.no_grad():
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x.cuda() for x in data_list[0]]
    sid_tgt1 = torch.LongTensor([1]).cuda()
    sid_tgt2 = torch.LongTensor([2]).cuda()
    sid_tgt3 = torch.LongTensor([4]).cuda()
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    audio2 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt2)[0][0,0].data.cpu().float().numpy()
    audio3 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt3)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt2.item())
ipd.display(ipd.Audio(audio2, rate=hps.data.sampling_rate, normalize=False))
print("Converted SID: %d" % sid_tgt3.item())
ipd.display(ipd.Audio(audio3, rate=hps.data.sampling_rate, normalize=False))